In [1]:
import os
import logging
import pandas as pd
import getpass
from datetime import datetime
logger = logging.getLogger('LabJournal')
logging.basicConfig(level=logging.DEBUG)

In [2]:
import sys
sys.path.append("../../../") # need this if labjournal is not installed

from simdb.utils.fileFinder import find_files
from simdb.databaseModel import *
from simdb.databaseAPI import store_dict
import simdb.utils.tpr_parser as tpr_parser


In [3]:
SIMS = '/home/andrejb/Research/SIMS'
GMXBIN="/home/soft/gromacs/gromacs-2018/inst/shared/bin/"
os.environ['GMXBIN'] = GMXBIN

user = getpass.getuser()

In [4]:
logger.info('create_database:create_database: FIND FILES')
ERRORS=False
WARNINGS=False

INFO:LabJournal:create_database:create_database: FIND FILES


In [6]:
%%time
SIM_IDS=[]
PATHS=[]
METAS=[]

for fname in find_files(pattern = 'topol.tpr', path = SIMS, dir_ignore = ['data']):
    path =  os.path.dirname(fname)
    sim_id = os.path.basename(path)
    try:
        meta = pd.Series.from_csv(os.path.join(path, "meta.csv"))
    except IOError:
        meta = pd.Series()
    SIM_IDS.append(sim_id)
    PATHS.append(path)
    METAS.append(meta)

INFO:Simulation_database:FileFinder: pattern: topol.tpr
INFO:Simulation_database:FileFinder: path: /home/andrejb/Research/SIMS
INFO:Simulation_database:FileFinder: dir_ignore: ['data']


CPU times: user 1.94 s, sys: 1.19 s, total: 3.13 s
Wall time: 8.43 s


In [8]:
db = 'andrej_raw.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

In [ ]:
%%time
for i, entry_id in enumerate(SIM_IDS):
    path = PATHS[i]
    tprfile = os.path.join(path,'topol.tpr')
    
    mapped_keywords = tpr_parser.main(tprfile)
    
    meta = METAS[i]
    try:
        description = meta['note']
        del meta['note']
    except KeyError:
        description = ""
    
    if 'created_on' in meta:
        created_on  = meta['created_on']
        del meta['created_on']
    else:
        created_on = datetime.fromtimestamp(os.path.getmtime(tprfile))
        
    sim = store_dict(
        entry_id    = entry_id,
        path        = path,
        sim_type    = "GROMACS",
        description = description,
        created_on  = created_on,
        owner       = user,
        raw_mdp_parameters = mapped_keywords,
        raw_keywords = meta,
    )
    
    session.add(sim)
    session.flush()

session.commit()
session.close()

In [8]:
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
session.query(Main).all()

/usr/local/lib/python2.7/dist-packages/sqlalchemy/sql/sqltypes.py:596: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


[Main(entry_id='2017_06_28_GfM_SMmin_rnd_k33_26', mediawiki='None', path='/home/andrejb/Research/SIMS/2017_06_28_GfM_SMmin_rnd_k33_26'),
 Main(entry_id='2017_06_09_GfM_SMmin_2ub_k11_01', mediawiki='None', path='/home/andrejb/Research/SIMS/2017_06_09_GfM_SMmin_2ub_k11_01'),
 Main(entry_id='2017_09_26_GfM_SMmin_rnd_k27_38', mediawiki='None', path='/home/andrejb/Research/SIMS/2017_09_26_GfM_SMmin_rnd_k27_38'),
 Main(entry_id='2017_06_09_GfM_SMmin_2ub_k48_01', mediawiki='None', path='/home/andrejb/Research/SIMS/2017_06_09_GfM_SMmin_2ub_k48_01'),
 Main(entry_id='2017_04_27_G_2ub_k27_01_06', mediawiki='None', path='/home/andrejb/Research/SIMS/2017_04_27_G_2ub_k27_01_06'),
 Main(entry_id='2017_02_14_G_2ub_k33_02_03', mediawiki='None', path='/home/andrejb/Research/SIMS/2017_02_14_G_2ub_k33_02_03'),
 Main(entry_id='2018_04_06_2xUb_unlinked_22', mediawiki='None', path='/home/andrejb/Research/SIMS/2018_04_06_2xUb_unlinked_22'),
 Main(entry_id='2017_09_26_GfM_SMmin_rnd_k33_36', mediawiki='None', p

In [10]:
session.close()

## Setup an empty DB

In [4]:
db = 'empty.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)
session.close()